# Assignment No.3 Part (C)

##  Introduction

There are several classical optimization algorithms provided by SciPy in the optimize package. An overview of the module is        below in the form of available optimization tools.

 
 

 ###  Optimization Tools  
 
#### A collection of general-purpose optimization routines.  
 
  fmin        --  Nelder-Mead Simplex algorithm  
                    (uses only function calls)  
  fmin_powell --  Powell’s (modified) level set method (uses only  
                    function calls)  
  fmin_bfgs   --  Quasi-Newton method (can use function and gradient)  
  fmin_ncg    --  Line-search Newton Conjugate Gradient (can use  
                    function, gradient and hessian).  
  leastsq     --  Minimize the sum of squares of M equations in  
                    N unknowns given a starting estimate.  
 
####  Scalar function minimizers  
 
  fminbound   --  Bounded minimization of a scalar function.  
  brent       --  1-D function minimization using Brent method.  
  golden      --  1-D function minimization using Golden Section method  
  bracket     --  Bracket a minimum (given two starting points)  
 
Also a collection of general_purpose root-finding routines.  
 
  fsolve      --  Non-linear multi-variable equation solver.  
 
 #### Scalar function solvers  
 
  brentq      --  quadratic interpolation Brent method  
  brenth      --  Brent method (modified by Harris with hyperbolic extrapolation)  
  ridder      --  Ridder’s method  
  bisect      --  Bisection method  
  newton      --  Secant method or Newton’s method   
  fixed_point -- Single-variable fixed-point solver.  

The first four algorithms are unconstrained minimization algorithms (fmin: Nelder-Mead simplex, fmin_bfgs: BFGS, fmin_ncg: Newton Conjugate Gradient, and leastsq: Levenburg-Marquardt). The fourth algorithm only works for functions of a single variable but allows minimization over a specified interval. The last algorithm actually finds the roots of a general function of possibly many variables. It is included in the optimization package because at the (non-boundary) extreme points of a function, the gradient is equal to zero. 


## Nelder-Mead Simplex algorithm (optimize.fmin)

The simplex algorithm is probably the simplest way to minimize a fairly well-behaved function. The simplex algorithm requires only function evaluations and is a good choice for simple minimization problems. However, because it does not use any gradient evaluations, it may take longer to find the minimum. To demonstrate the minimization function consider the problem of minimizing the Rosenbrock function of N variables: 
 
The minimum value of this function is 0 which is achieved when xi = 1. This minimum can be found using the fmin routine as shown in the example below: 


In [1]:
>>> from scipy.optimize import fmin  
>>> def rosen(x):  # The Rosenbrock function  
        return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)  
 
>>> x0 = [1.3, 0.7, 0.8, 1.9, 1.2]  
>>> xopt = fmin(rosen, x0)  


Optimization terminated successfully.
         Current function value: 0.000066
         Iterations: 141
         Function evaluations: 243


## Broyden-Fletcher-Goldfarb-Shanno algorithm (optimize.fmin_bfgs)

In order to converge more quickly to the solution, this routine uses the gradient of the objective function. If the gradient is not given by the user, then it is estimated using first-differences. The Broyden-Fletcher-Goldfarb-Shanno (BFGS) method typically requires fewer function calls than the simplex algorithm even when the gradient must be estimated. 
To demonstrate this algorithm, the Rosenbrock function is again used. The gradient of the Rosenbrock function is the vector: 
 
This expression is valid for the interior derivatives. Special cases are
 
A Python function which computes this gradient is constructed by the code-segment: 


In [11]:
>>> def rosen_der(x):  
        xm = x[1:-1]  
        xm_m1 = x[:-2]  
        xm_p1 = x[2:]  
        der = zeros(x.shape,x.typecode())  
        der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)  
        der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])  
        der[-1] = 200*(x[-1]-x[-2]**2)    
>>> from scipy.optimize import fmin_bfgs  
 
>>> x0 = [1.3, 0.7, 0.8, 1.9, 1.2]  
>>> xopt = fmin_bfgs(rosen, x0, fprime=rosen_der)  


NameError: name 'zeros' is not defined

## Newton-Conjugate-Gradient (optimize.fmin_ncg)

The method which requires the fewest function calls and is therefore often the fastest method to minimize functions of many variables is fmin_ncg. This method is a modified Newton’s method and uses a conjugate gradient algorithm to (approximately) invert the local Hessian. Newton’s method is based on fitting the function locally to a quadratic form: 
 
where H  is a matrix of second-derivatives (the Hessian). If the Hessian is positive definite then the local minimum of this function can be found by setting the gradient of the quadratic form to zero, resulting in 
 
The inverse of the Hessian is evaluted using the conjugate-gradient method. An example of employing this method to minimizing the Rosenbrock function is given below. To take full advantage of the NewtonCG method, a function which computes the Hessian must be provided. The Hessian matrix itself does not need to be constructed, only a vector which is the product of the Hessian with an arbitrary vector needs to be available to the minimization routine. As a result, the user can provide either a function to compute the Hessian matrix, or a function to compute the product of the Hessian with an arbitrary vector. 


## Least-square fitting (minimize.leastsq)

All of the previously-explained minimization procedures can be used to solve a least-squares problem provided the appropriate objective function is constructed.

An example should clarify the usage. Suppose it is believed some measured data follow a sinusoidal pattern 
 
where the parameters A, k, and θ are unknown. The residual vector is 
 
By defining a function to compute the residuals and (selecting an appropriate starting position), the least-squares fit routine can be used to find the best-fit parameters Â,  ,  . This is shown in the following:



In [45]:
>>> import numpy
>>> x = arange(0,6e-2,6e-2/30)  
>>> A,k,theta = 10, 1.0/3e-2, pi/6  
>>> y_true = A*sin(2*pi*k*x+theta)  
>>> y_meas = y_true + 2*randn(len(x))  
 
>>> def residuals(p, y, x):  
        A,k,theta = p  
        err = y-A*sin(2*pi*k*x+theta)  
        return err  
 
>>> def peval(x, p):  
        return p[0]*sin(2*pi*p[1]*x+p[2])  
 
>>> p0 = [8, 1/2.3e-2, pi/3]  
>>> print (array(p0))   
 
>>> from scipy.optimize import leastsq  
>>> plsq = leastsq(residuals, p0, args=(y_meas, x))  
>>> print (plsq[0])  
 
>>> print (array([A, k, theta]))  
 




NameError: name 'arange' is not defined

## Scalar function minimizers

Often only the minimum of a scalar function is needed (a scalar function is one that takes a scalar as input and returns a scalar output). In these circumstances, other optimization techniques have been developed that can work faster. 

### Unconstrained minimization (optimize.brent)

There are actually two methods that can be used to minimize a scalar function (brent and golden), but golden is included only for academic purposes and should rarely be used. The brent method uses Brent’s algorithm for locating a minimum. Optimally a bracket should be given which contains the minimum desired. A bracket is a triple  such that f  > f  < f  and a < b < c. If this is not given, then alternatively two starting points can be chosen and a bracket will be found from these points using a simple marching algorithm. If these two starting points are not provided 0 and 1 will be used (this may not be the right choice for your function and result in an unexpected minimum being returned). 

### Bounded minimization (optimize.fminbound)

Thus far all of the minimization routines described have been unconstrained minimization routines. Very often, however, there are constraints that can be placed on the solution space before minimization occurs. The fminbound function is an example of a constrained minimization procedure that provides a rudimentary interval constraint for scalar functions. The interval constraint allows the minimization to occur only between two fixed endpoints. 
For example, to find the minimum of J1  near x = 5, fminbound can be called using the interval  as a constraint. The result is xmin = 5.3314: 


In [28]:
>>> from scipy.special import j1  
 
>>> from scipy.optimize import fminbound  
>>> xmin = fminbound(j1, 4, 7)  
>>> print xmin   
  


SyntaxError: Missing parentheses in call to 'print' (<ipython-input-28-e2c5fe1684f1>, line 5)

## Root finding 

### Sets of equations

To find the roots of a polynomial, the command roots is useful. To find a root of a set of non-linear equations, the command optimize.fsolve is needed. For example, the following example finds the roots of the single-variable transcendental equation 
 
and the set of non-linear equations
 
The results are x = -1.0299 and x0 = 6.5041, x1 = 0.9084. 


In [29]:
>>> def func(x):  
        return x + 2*cos(x)  
 
>>> def func2(x):  
        out = [x[0]*cos(x[1]) - 4]  
        out.append(x[1]*x[0] - x[1] - 5)  
        return out  
 
>>> from optimize import fsolve  
>>> x0 = fsolve(func, 0.3)  
>>> print x0  
-1.02986652932  
 
>>> x02 = fsolve(func2, [1, 1])  
>>> print x02  
[ 6.5041  0.9084] 


SyntaxError: Missing parentheses in call to 'print' (<ipython-input-29-2a54ff499c37>, line 11)

### Scalar function root finding

If one has a single-variable equation, there are four different root finder algorithms that can be tried. Each of these root finding algorithms requires the endpoints of an interval where a root is suspected (because the function changes signs). In general brentq is the best choice, but the other methods may be useful in certain circumstances or for academic purposes. 

### Fixed-point solving

A problem closely related to finding the zeros of a function is the problem of finding a fixed-point of a function. A fixed point of a function is the point at which evaluation of the function returns the point: g  = x. Clearly the fixed point of g is the root of f  = g  - x. Equivalently, the root of f is the fixed_point of g  = f  + x. The routine fixed_point provides a simple iterative method using Aitkens sequence acceleration to estimate the fixed point of g given a starting point. 

